In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path
from src.utils.data import ensure_synthetic_data, load_prices_csv, pivot_close, train_test_split_time
from src.strategies.baseline import sma_crossover
from src.backtest.engine import SingleAssetBacktester, BacktestConfig
from src.evaluation.report import summarize_single, summarize_portfolio
from src.models.econometric import arima_forecast
from src.models.ml import MLConfig, train_ml_model, to_positions
from src.agents.risk_guardian import RiskGuardian
from src.agents.pm_agent import PMAgent
from src.portfolio.optimizers import mean_variance_opt, hrp_weights

ROOT = Path(".")
ensure_synthetic_data(ROOT)
ART = Path("notebooks/artifacts"); ART.mkdir(parents=True, exist_ok=True)
DATA_SMALL = Path("data/prices_small.csv"); DATA_LARGE = Path("data/prices_large.csv")
SEED = 123; np.random.seed(SEED)

def plot_equity(eq: pd.Series, title: str, out: Path):
    plt.figure(figsize=(8,4)); plt.plot(eq.index, eq.values); plt.title(title)
    plt.xlabel("Date"); plt.ylabel("Equity"); plt.grid(True, alpha=0.3)
    plt.tight_layout(); plt.savefig(out, dpi=150); plt.close()

In [ ]:
# ===================== Level 1: Baseline (single BTC/USDT) =====================
df = load_prices_csv(DATA_SMALL)
btc = df[df["symbol"]=="BTCUSDT"].set_index("date")["close"].astype(float)
pos = sma_crossover(btc, fast=20, slow=60)
bt = SingleAssetBacktester(btc, BacktestConfig(fee_bps=5, slippage_bps=5, initial_equity=100000.0))
res = bt.run(pos)
rpt1 = summarize_single(res["equity"], res["returns"])
plot_equity(res["equity"], "Level 1 — SMA Baseline (BTCUSDT)", ART/"level1_equity.png")
print("Level 1:", rpt1)

In [ ]:
# ===================== Level 2: Econometric, ML, AI Agent =====================
# Econometric (ARIMA or momentum fallback)
ar_pred = arima_forecast(btc, order=(1,1,1), horizon=1)
res_ar = bt.run(to_positions(ar_pred.reindex(btc.index).fillna(0.0)))
rpt2_ar = summarize_single(res_ar["equity"], res_ar["returns"])
plot_equity(res_ar["equity"], "Level 2 — Econometric (ARIMA)", ART/"level2_arima_equity.png")

# ML (GBR)
ml_out = train_ml_model(btc, MLConfig(target_horizon=1, lags=20, random_state=SEED))
pred_gbr = ml_out["pred_test"]["gbr"]
res_ml = bt.run(to_positions(pred_gbr.reindex(btc.index).fillna(0.0)))
rpt2_ml = summarize_single(res_ml["equity"], res_ml["returns"])
plot_equity(res_ml["equity"], "Level 2 — ML (GBR)", ART/"level2_ml_equity.png")

# AI Agent over ML positions (RiskGuardian scaling)
guardian = RiskGuardian(alpha=0.9, max_leverage=1.0)
pos_ml_guard = guardian.adjust_positions(to_positions(pred_gbr.reindex(btc.index).fillna(0.0)), btc)
res_ml_guard = bt.run(pos_ml_guard)
rpt2_guard = summarize_single(res_ml_guard["equity"], res_ml_guard["returns"])
plot_equity(res_ml_guard["equity"], "Level 2 — AI Agent (RiskGuardian)", ART/"level2_agent_equity.png")

print("Level 2 — ARIMA:", rpt2_ar)
print("Level 2 — ML(GBR):", rpt2_ml)
print("Level 2 — Agent(ML+Guardian):", rpt2_guard)

In [ ]:
# ===================== Level 3: Static Portfolio (5–7 coins) ==================
syms = ["BTCUSDT","ETHUSDT","BNBUSDT","SOLUSDT","ADAUSDT","XRPUSDT","MATICUSDT"]
wide = pivot_close(df[df["symbol"].isin(syms)])
ret = wide.pct_change().dropna()
mu, cov = ret.mean()*252.0, ret.cov()*252.0

w_mv = mean_variance_opt(mu, cov, lmbd=10.0)
w_hrp = hrp_weights(ret)
# BL via PMAgent (views = 60D momentum annualized)
views = (wide.pct_change(60).iloc[-1]).dropna() * 6.0
pm = PMAgent(method="bl", risk_aversion=10.0, bl_tau=0.05)
w_bl = pm.allocate(wide, views=views)

def backtest_static_weights(ret, w, title, out_png):
    eq = (1 + ret @ w).cumprod() * 100000
    rpt = summarize_portfolio(eq, (ret @ w))
    plot_equity(eq, title, out_png); print(title, rpt)

backtest_static_weights(ret, w_mv, "Level 3 — MV", ART/"level3_mv_equity.png")
backtest_static_weights(ret, w_hrp, "Level 3 — HRP", ART/"level3_hrp_equity.png")
backtest_static_weights(ret, w_bl, "Level 3 — BL", ART/"level3_bl_equity.png")

In [ ]:
# ===================== Level 4: Dynamic Rebalancing (monthly BL) =============
monthly = pd.date_range(wide.index[-365], wide.index[-1], freq="30D").intersection(wide.index)
weights_over_time = []
for dt in monthly:
    hist = wide[wide.index <= dt].tail(252)
    if len(hist) < 60: continue
    views = hist.pct_change(60).iloc[-1].dropna() * 6.0
    weights_over_time.append(pd.Series(pm.allocate(hist, views=views), name=dt))
weights_over_time = pd.DataFrame(weights_over_time).fillna(0.0)

ret_last_year = ret.loc[weights_over_time.index.min():]
port_ret = []
cur_w = weights_over_time.iloc[0]
for dt, row in ret_last_year.iterrows():
    if dt in weights_over_time.index: cur_w = weights_over_time.loc[dt].fillna(0.0)
    port_ret.append((row * cur_w).sum())
port_ret = pd.Series(port_ret, index=ret_last_year.index)
eq4 = (1 + port_ret).cumprod() * 100000
rpt4 = summarize_portfolio(eq4, port_ret, weights=weights_over_time)
plot_equity(eq4, "Level 4 — Dynamic BL (Monthly)", ART/"level4_dynamic_equity.png")
print("Level 4:", rpt4)

# ===================== Level 5: 100+ coins dynamic BL ========================
df_large = load_prices_csv(DATA_LARGE)
df_large["dollar_vol"] = df_large["close"] * df_large["volume"]
top100 = df_large.groupby("symbol")["dollar_vol"].mean().sort_values(ascending=False).head(100).index.tolist()
wide100 = pivot_close(df_large[df_large["symbol"].isin(top100)])
ret100 = wide100.pct_change().dropna()

def momentum_views(wide_df, lookback=60):
    mom = wide_df.pct_change(lookback).iloc[-1]
    vol = wide_df.pct_change().rolling(20).std().iloc[-1]
    conf = 1.0 / (vol + 1e-6)
    vw = (mom * conf).fillna(0.0)
    return vw / (vw.abs().mean() + 1e-6) * 0.1

pm_large = PMAgent(method="bl", risk_aversion=12.0, bl_tau=0.03)
monthly100 = pd.date_range(wide100.index[-365], wide100.index[-1], freq="30D").intersection(wide100.index)

weights_over_time_100 = []
for dt in monthly100:
    hist = wide100[wide100.index <= dt].tail(252)
    if len(hist) < 60: continue
    vw = momentum_views(hist, 60)
    w = pm_large.allocate(hist, views=vw)
    dv = (df_large[df_large["date"]==dt].set_index("symbol")["dollar_vol"] if (df_large["date"]==dt).any()
          else df_large.groupby("symbol")["dollar_vol"].mean())
    dv = dv.reindex(w.index).fillna(dv.mean())
    cap = (dv.rank(pct=True) >= 0.1).astype(float)  # drop bottom 10% liquidity
    w = (w * cap); w = w / (w.abs().sum() + 1e-12)
    weights_over_time_100.append(pd.Series(w, name=dt))
weights_over_time_100 = pd.DataFrame(weights_over_time_100).fillna(0.0)

ret_last_year_100 = ret100.loc[weights_over_time_100.index.min():]
port_ret_100 = []
cur_w = weights_over_time_100.iloc[0]
for dt, row in ret_last_year_100.iterrows():
    if dt in weights_over_time_100.index: cur_w = weights_over_time_100.loc[dt].fillna(0.0)
    port_ret_100.append((row * cur_w).sum())
port_ret_100 = pd.Series(port_ret_100, index=ret_last_year_100.index)
eq5 = (1 + port_ret_100).cumprod() * 100000
rpt5 = summarize_portfolio(eq5, port_ret_100, weights=weights_over_time_100)
plot_equity(eq5, "Level 5 — Dynamic BL (100+)", ART/"level5_dynamic_equity.png")
print("Level 5:", rpt5)

print("\nAll levels executed. Artifacts saved under notebooks/artifacts/")